In [31]:
from openai import OpenAI
import os
import json
import re
from key import get_key

In [32]:
client = OpenAI(
    api_key= get_key()
)

In [33]:
# XML folder
OLD_XML_DIR = "../layouts/v6.1.0"
NEW_XML_DIR = "../layouts/v6.3.0"
OLD_TESTS_DIR = "../tests/v6.1.0"
NEW_TESTS_DIR = "../tests/v6.3.0"
NEW_VERSION = "v6.3.0"

In [34]:
# Read file content
def read_file(file_path):
    with open(file_path, "r", encoding="utf-8") as file:
        return file.read()

# Get list of files
old_xml_files = {f: read_file(os.path.join(OLD_XML_DIR, f)) for f in os.listdir(OLD_XML_DIR)}
new_xml_files = {f: read_file(os.path.join(NEW_XML_DIR, f)) for f in os.listdir(NEW_XML_DIR)}
old_test_files = {f: read_file(os.path.join(OLD_TESTS_DIR, f)) for f in os.listdir(OLD_TESTS_DIR)}

In [35]:
# Convert XMLs into a single string
old_xml_data = "\n\n".join([f"File: {name}\n{content}" for name, content in old_xml_files.items()])
new_xml_data = "\n\n".join([f"File: {name}\n{content}" for name, content in new_xml_files.items()])

In [36]:
# === STEP 1: Detect UI Changes ===
xml_differences = {}

for xml_name, old_xml_content in old_xml_files.items():
    new_xml_content = new_xml_files.get(xml_name, "")

    # Construct LLM prompt for detecting differences
    prompt = f"""
    You are an expert in analyzing Android UI XML layouts.
    Here is the old UI XML layout:
    {old_xml_content}

    Here is the updated UI XML layout:
    {new_xml_content}

    Please list all UI changes between these two versions.
    Format the response as structured text, including:
    - Elements that were **added** (with their IDs and attributes).
    - Elements that were **removed**.
    - Elements that were **renamed**.
    - Any **structural changes** (e.g., button moved inside another layout).

    Return the response in JSON format.
    """

    response = client.chat.completions.create(
        messages=[{"role": "system", "content": "You are an expert in Android UI structure analysis."},
                  {"role": "user", "content": prompt}],
        model="gpt-4o-mini"
    )

    xml_differences[xml_name] = response.choices[0].message.content

    print(f"XML differences detected for {xml_name}")

XML differences detected for screen7.xml
XML differences detected for screen6.xml
XML differences detected for screen5.xml
XML differences detected for screen1.xml
XML differences detected for screen2.xml
XML differences detected for screen3.xml
XML differences detected for screen12.xml
XML differences detected for screen11.xml
XML differences detected for screen10.xml
XML differences detected for screen14.xml
XML differences detected for screen15.xml
XML differences detected for screen8.xml
XML differences detected for screen9.xml


In [37]:
def clean_java_code(java_code):
    """ Remove the code block markers (```java ... ```) """
    return re.sub(r'```java\n|\n```', '', java_code)

for test_name, old_test in old_test_files.items():
    prompt = f"""
    You are an expert in Android UI test automation using Espresso.

    The Android app's UI has changed, and test cases need to be updated accordingly.

    Here are all the detected UI changes:
    {all_differences}

    Here is an old test case:
    {old_test}

    Please update the test case to ensure it remains valid with the new UI structure.
    Ensure that:
    - It still verifies the original functionality.
    - It interacts with the correct elements based on the updated XML.
    - Any removed elements are handled gracefully.
    - Any renamed or moved elements are adjusted accordingly.
    - It follows best practices for Android UI test automation.

    Return only the updated Java test code without any explanation.
    """

    response = client.chat.completions.create(
        messages=[{"role": "system", "content": "You are an expert in Android UI test automation."},
                  {"role": "user", "content": prompt}],
        model="gpt-4o-mini"
    )

    updated_java_code = response.choices[0].message.content.strip()
    cleaned_java_code = clean_java_code(updated_java_code)

    # Save the cleaned Java test as a .java file
    file_path = f"../tests/{NEW_VERSION}/{test_name}"
    with open(file_path, "w") as f:
        f.write(cleaned_java_code)
    print(f"Saved {file_path}")

Saved ../tests/v6.3.0/CategoryLifecycleTest.java
Saved ../tests/v6.3.0/SearchTagBackArrowTest.java
Saved ../tests/v6.3.0/NoteLifecycleTest.java
Saved ../tests/v6.3.0/FabCameraNoteTest.java
Saved ../tests/v6.3.0/AutoBackupTest.java
Saved ../tests/v6.3.0/RecurrenceRuleTest.java
Saved ../tests/v6.3.0/SettingsActivityTest.java
Saved ../tests/v6.3.0/RemindersLifecycleTest.java
Saved ../tests/v6.3.0/BaseEspressoTest.java
Saved ../tests/v6.3.0/MrJingleLifecycleTest.java
Saved ../tests/v6.3.0/NoteListMenuTest.java
Saved ../tests/v6.3.0/DrawerMenusEspressoTest.java
Saved ../tests/v6.3.0/FabLifecycleTest.java
